In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option("display.max_columns",None)

# 1. Exploratory Data Analysis

In [ ]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.shape

(768, 9)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [ ]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

# 2. Data Preprocessing & Feature Engineering

In [ ]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis = 1)

In [ ]:
#Uzaklık temelli ve Gredient Descent gibi yöntemlerde değişkenlerin standart olması,
# elde edilecek sonuçların ya daha hızlı ya da daha doğru olmasını sağlayacaktır.
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

* Standart Scaler : Farklı birimlere karşılık gelen verileri karşılaştırmak istediğinizde bu kullanışlıdır. Bu durumda, birimleri kaldırmak istiyorsunuz. Bunu, tüm verileri tutarlı bir şekilde yapmak için, verileri varyansın üniter olduğu ve serinin ortalamanın 0 olduğu şekilde dönüştürürsünüz.

In [ ]:
#Bu bize bir array döndürüyor ama süün isimleri yok. Bunun için :
#Ölçeklendirilmiş ve standartlaştırılmış bağımsız değişken değerleri :
X = pd.DataFrame(X_scaled, columns=X.columns)
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


# 3. Modeling & Prediction

In [ ]:
#KNN ile modelimizi kurduk.
knn_model = KNeighborsClassifier().fit(X,y)
knn_model

KNeighborsClassifier()

In [ ]:
#Kurduğumuz modele rastgele bir user test edebilmek için random_user oluşturuyoruz.
random_user = X.sample(1, random_state=45) #xden bir tane seç
random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
195,0.342981,1.161295,0.770014,1.283638,1.130518,0.940144,-0.232176,-0.360847


NOT : 
* Train_test_split, dizileri veya matrisleri rasgele tren ve test alt kümelerine böler. Bu, random_state belirtmeden her çalıştırışınızda farklı bir sonuç alacağınız, bunun beklenen davranış olduğu anlamına gelir. Gerçek random_state numarasının ne olduğu önemli değildir.  Önemli olan, number_state e atadığınız sayıyı her kullandığınızda, bölmeyi ilk yaptığınızda daima aynı çıktıyı elde edersiniz. Bu, örneğin dokümantasyonda tekrarlanabilir sonuçlar almak istediğinizde faydalıdır, böylece örnekleri çalıştırırken herkes aynı numaraları görebilir. Pratikte şunu söyleyeyim, bir şeyleri test ederken random_state dosyasını bir sabit sayıya ayarlamanız gerekir, ancak daha sonra rastgele (ve sabit olmayan) bir bölünmeye ihtiyacınız varsa, bunu üretimde kaldırmalısınız.

In [ ]:
#Bir gözlem birimi için tahminde bulunduk.
knn_model.predict(random_user)

array([1])

# 4. Model Başarısını Değerlendirme

In [ ]:
#Bütün gözlem değerleri için tahminde bulunduk. Confusion matrix için y_pred:
y_pred = knn_model.predict(X)

In [ ]:
#AUC için y_prob:
y_prob = knn_model.predict_proba(X)[:,1] #[:,1] bir sınıfına ait olma olasılıklarını hesaplıyoruz.
#Bunun üzerinden roc eğrisini hesaplıyor olucaz.

In [ ]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



* PRECISION = 0.79 -> 1 sınıfına yönelik  tahmin ettiklerimizin başarısı
* RECALL = 0.70 -> gerçekte 1 olanları 1 olarak tahmin etme başarımız
* F1-SCORE = 0.74 -> bunların (recall-precision) harmonik ortalaması
* ACCURACY = 0.83 -> tahmin başarımız

In [ ]:
#AUC
roc_auc_score(y, y_prob)

0.9017686567164179

* AUC = 0.90 maşallah baya iyi tahmin ediyoruz. Ama dengesiz veri dağılımını da göz önünde bulundurmak lazım.
* Ayrıca modeli kurduğumuz  veri ile test ettik. Aslında yapılması gereken şey modelin görmediği verideki performansını değerlendirmektir. (holdout, crossvalidation)

In [ ]:
#cross_validate işlevi cross_val_score'dan iki şekilde farklıdır: Değerlendirme için birden çok metrik belirtmeye izin verir.(örneğin: scoring=["accuracy", "f1", "roc_auc"])
cv_result = cross_validate(knn_model, X, y, cv=5,scoring=["accuracy", "f1", "roc_auc"])
cv_result

{'fit_time': array([0.00517869, 0.00425673, 0.00428772, 0.00428343, 0.00468731]),
 'score_time': array([0.02256727, 0.02260208, 0.02169085, 0.02246904, 0.02601099]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [ ]:
#Şuan cross_validate veri setimizi cv=5 ile belirttiğimizden 5e böldü, 4yle model geliştirdi 1yle de test etti.
#Yukarıda görüldüğü gibi her bir değer farklı, bunların ortalamasını alarak bütün test scorlarının ortalamasını almış olucaz.
cv_result["test_accuracy"].mean()

0.733112638994992

In [ ]:
cv_result["test_f1"].mean()

0.5905780011534191

In [ ]:
cv_result["test_roc_auc"].mean()

0.7805279524807827

* ACCURACY = 0.73 -> diğer accuracy (83)değerimizle arasında 10 birim fark var.
* AUC = 0.78 -> diğer auc değerimizle(90) arasında 12 birim fark var.
* F1 = 0.59 -> diğer auc değerimizle (74)arasında 15 birim fark var.

Buradan da anlayacağımız gibi cross_validation yöntemiyle verilerimizi modelleyip test ettiğimizde bütün veriyle model kurmaya kıyasla daha farklı ve bana kalırsa daha güvenilir sonuçlar aldık.


**PEKİ BAŞARI SONUÇLARIMIZ NASIL ARTTIRILABİLİR?**
> * Örnek boyutu arttırılabilir.
> * Veri ön işleme, işlemleri detaylandırabilir.
> * Özellik mühendisliği (yeni değişkenler türetilebilir.)
> * İlgili algoritma için optimizasyonlar yapılabilir. (hyperparametreler değiştirilebilir.)



In [ ]:
#Hyperparametreler: ne olması gerektiği, modeli tasarlayan kişiye bırakılmış, probleme, veri setine göre değişiklik gösteren parametrelere denir.
#Parametreler ise veri içerisinden öğrenilmektedir.
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

# 5. Hyperparameter Optimizasyonu


In [ ]:
#Hyperparametreler: ne olması gerektiği, modeli tasarlayan kişiye bırakılmış, probleme, veri setine göre değişiklik gösteren parametrelere denir.
knn_model = KNeighborsClassifier()
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
#n_neighbors yani komşu sayısını değiştirerek model başarısını daha da iyileştirmek istiyoruz.
knn_params = {"n_neighbors": range(2,50)} #2-50 arasında sayılar oluşturarak bunları n_neighbors da deneyeceğiz.
# GridSearchCV -> Izgarada cross val ile ara anlamında.
# GridSearchCV der ki bana modelini göster(knn_model), hangi parametre setini denemek istiyorsun onu göster(knn_params), 
# kaç katlı cv yapacaksın göster(cv=5), n_jobs =-1 yapılması durumunda işlemcileri tam performansında kullanır,
# verbose ise bir raporlama isteyip istemediğimize göre 1-0 verilir(verbose=1). Biz istiyoruz. 
knn_gs_best = GridSearchCV(knn_model, knn_params, cv=5, n_jobs=-1, verbose = 1).fit(X,y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
#Ön tanımlı n_neighbors umuz 5 idi. Burada gördüğümüz üzere n_neighborsu 17 yapmamız bizi en iyi sonuca ulaştırabilir.
knn_gs_best.best_params_

{'n_neighbors': 17}

# 6. Final Model

In [ ]:
#** kullanmamızın sebebi knn_gs_params.best_params_ içerisinde şimdilik tek bir değer var ama bu ileride birden fazla değer olmayacağı anlamına gelmiyor.
#Tek tek elimizle belirtemeyeceğimiz için ** kullanıyoruz. (**kwargs) anahtar sözcüklü , değişken uzunluklu bir bağımsız değişken listesini iletmek için kullanılır.
# https://www.geeksforgeeks.org/args-kwargs-python/
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X,y)

In [ ]:
cv_result = cross_validate(knn_final, X, y, cv=5,scoring=["accuracy", "f1", "roc_auc"])
cv_result

{'fit_time': array([0.00706387, 0.00422812, 0.00416589, 0.00463295, 0.00517821]),
 'score_time': array([0.02467728, 0.02488256, 0.02492738, 0.03744793, 0.02492762]),
 'test_accuracy': array([0.75324675, 0.74025974, 0.75974026, 0.81045752, 0.77124183]),
 'test_f1': array([0.60416667, 0.60784314, 0.57471264, 0.70103093, 0.59770115]),
 'test_roc_auc': array([0.80675926, 0.7912037 , 0.79166667, 0.84603774, 0.82830189])}

In [ ]:
cv_result["test_accuracy"].mean()

0.7669892199303965

In [ ]:
cv_result["test_f1"].mean()

0.6170909049720137

In [ ]:
cv_result["test_roc_auc"].mean()

0.8127938504542278

* ACCURACY = 0.76 -> accuracy değerimiz artmış, bundan bir önceki modeldeki accuracy değerimiz 0.73 
* AUC = 0.81 -> auc değerimiz artmış, bundan bir önceki modeldeki auc değerimiz 0.78 
* F1 = 0.61  -> f1 değerimiz artmış, bundan bir önceki modeldeki f1 değerimiz 0.59 

In [ ]:
random_user = X.sample(1)
random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
224,-0.844885,-0.653939,-0.160546,-0.347291,-0.206648,-1.06518,0.586277,-0.616111


In [ ]:
#Elde ettiğimiz random biri üzerinden bir tahminde bulunmak istersek :
knn_final.predict(random_user)

array([0])